In [1]:
import torchaudio
import torch 
import pandas as pd
import numpy as np
import re 
import os 
from torch.nn import functional as F
import random
import math
import pickle
import gc
import torch.nn as nn
from tqdm.notebook import tqdm
import os
import glob
import warnings
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings(action='ignore')

In [2]:
base_dir = '/home/work/'
np1 = glob.glob(base_dir+'mel_dataset1/*.npy')
np2 = glob.glob(base_dir+'mel_dataset2/*.npy')
np3 = glob.glob(base_dir+'mel_dataset3/*.npy')
np4 = glob.glob(base_dir+'mel_dataset4/*.npy')
np5 = glob.glob(base_dir+'mel_dataset5/*.npy')
np6 = glob.glob(base_dir+'mel_dataset6/*.npy')
np7 = glob.glob(base_dir+'mel_dataset7/*.npy')
np8 = glob.glob(base_dir+'mel_dataset8/*.npy')

In [3]:
total = np1+np2+np3+np4+np5+np6+np7+np8

In [4]:
train_csv = pd.read_csv('new_train.csv')
valid_csv = pd.read_csv('new_valid.csv')
all_csv =pd.read_csv('new_all.csv')

In [5]:
def npy_list(csv):
    npy_list = []
    for song_path in tqdm(csv['npy_path']):
        song_npy = np.load(song_path).squeeze()
        #npy_list.append(song_npy[-1876:])
        npy_list.append(song_npy)
        
    return npy_list

train_list = [np.load(song_path).squeeze() for song_path in tqdm(train_csv['npy_path'])]
valid_list = [np.load(song_path).squeeze() for song_path in tqdm(valid_csv['npy_path'])]
#train_list = npy_list(train_csv)
#valid_list = npy_list(valid_csv)


  0%|          | 0/3825 [00:00<?, ?it/s]

  0%|          | 0/675 [00:00<?, ?it/s]

In [32]:
#all_list = train_list + valid_list

In [6]:

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(DEVICE)


cuda


In [7]:

# epoch문 안에 들어가야되나?
#train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
#val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)


In [7]:
def train(model, train_loader):
    model.train()
    train_loss = 0
    for batch in train_loader: 
        
        
        mel = torch.FloatTensor(batch).to(DEVICE)
    
        optimizer.zero_grad()
        
        encode, output = model(mel)
        
        loss = criterion(output, mel)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
    train_loss /= (len(train_loader))

    return train_loss

def val(model, train_loader):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in train_loader:
            
            
            mel = torch.FloatTensor(batch).to(DEVICE)
            
            encode, output = model(mel)

            loss = criterion(output, mel)
            
            val_loss += loss.item()

    val_loss /= (len(train_loader))

    return val_loss


def get_mel_embeding(model, train_loader):
    model.eval()
    mel_embeding_li = []
    with torch.no_grad():
        for batch in train_loader: 
            
            mel = torch.FloatTensor(batch).to(DEVICE)
            
            encode, output = model(mel)
            
            mel_embeding_li.append(encode.detach().cpu().numpy())

    return mel_embeding_li

In [8]:

############ 원래 ##############
# encoder <- 과거의 데이터를 바탕으로 미래를 예측 (t-1의 인과성 학습)
# decoder <- 미래의 데이터를 바탕으로 과거를 예측 (t+1의 인과성 학습)
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 128),
            #nn.BatchNorm1d(128),
            nn.Tanh(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(128, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 16, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 1024, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 1)
        )

        self.concat7_shape = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            
        )
        
        self.concat6_shape = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            
        )
        
        self.concat5_shape = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            
        )
        
        self.concat4_shape = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            
        )
        
        self.concat3_shape = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            
        )
        
        self.concat2_shape = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
        )
        
        self.concat1_shape = nn.Sequential(
            nn.Conv1d(in_channels = 1024, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            
        )
        
        
    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x1 = self.conv1(x)
        #print(x1.shape)
        x = F.pad(x1, pad = (4, 0, 0, 0))
        x2 = self.conv2(x)
        #print(x2.shape)
        x = F.pad(x2, pad = (8, 0, 0, 0))
        x3 = self.conv3(x)
        #print(x3.shape)
        x = F.pad(x3, pad = (16, 0, 0, 0))
        x4 = self.conv4(x)
        #print(x4.shape)
        x = F.pad(x4, pad = (32, 0, 0, 0))
        x5 = self.conv5(x)
        #print(x5.shape)
        x = F.pad(x5, pad = (64, 0, 0, 0))
        x6 = self.conv6(x)
        #print(x6.shape)
        x = F.pad(x6, pad = (128, 0, 0, 0))
        x7 = self.conv7(x)
        #print(x7.shape)
        encode = self.encoder_fc(x7.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        x = torch.cat([x7,x],1) # 4,16,7501
        #x = self.concat7_shape(x) 
        x = F.pad(x, pad = (128, 0, 0, 0)) 
        x = self.t_conv1(x) # 4,8,7501
        #print(x.shape,'new_cat7')
        
        
        x = torch.cat([x6,x],1)
        #x = self.concat6_shape(x)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.t_conv2(x)
        
        
        x = torch.cat([x5,x],1)
        #x = self.concat5_shape(x)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.t_conv3(x)
        
        
        
        x = torch.cat([x4,x],1)
        #x = self.concat4_shape(x)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.t_conv4(x)
        
        
        x = torch.cat([x3,x],1)
        #x = self.concat3_shape(x)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.t_conv5(x)
        
        
        
        x = torch.cat([x2,x],1)
        #x = self.concat2_shape(x)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv6(x)
        
        
        
        x = torch.cat([x1,x],1)
        #x = self.concat1_shape(x)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv7(x)
        
        #print(x.shape)
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x


In [58]:
'''
# no dilation
# encoder <- 과거의 데이터를 바탕으로 미래를 예측 (t-1의 인과성 학습)
# decoder <- 미래의 데이터를 바탕으로 과거를 예측 (t+1의 인과성 학습)
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 128),
            nn.BatchNorm1d(128),
            nn.Tanh(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(128, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 512, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 1)
        )

    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x = self.conv1(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.conv2(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.conv3(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.conv4(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.conv5(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.conv6(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.conv7(x)
        #print(x.shape)
        encode = self.encoder_fc(x.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv1(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv2(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv3(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv4(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv5(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv6(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv7(x)
        #print(x.shape)
        x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x
'''

"\n# no dilation\n# encoder <- 과거의 데이터를 바탕으로 미래를 예측 (t-1의 인과성 학습)\n# decoder <- 미래의 데이터를 바탕으로 과거를 예측 (t+1의 인과성 학습)\nclass TimeAutoEncoder(nn.Module):\n    def __init__(self):\n        super(TimeAutoEncoder, self).__init__()\n        self.conv1 = nn.Sequential(\n            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),\n            nn.BatchNorm1d(512),\n            nn.ReLU(),\n        )\n\n        self.conv2 = nn.Sequential(\n            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 1),\n            nn.BatchNorm1d(256),\n            nn.ReLU(),\n        )\n\n        self.conv3 = nn.Sequential(\n            nn.Conv1d(in_channels = 256, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 1),\n            nn.BatchNorm1d(128),\n            nn.ReLU(),\n        )\n\n        self.conv4 = nn.Sequential(\n            nn.Conv1d(in_channels = 128, out_channels = 64

In [59]:
'''
# skip connection
# encoder <- 과거의 데이터를 바탕으로 미래를 예측 (t-1의 인과성 학습)
# decoder <- 미래의 데이터를 바탕으로 과거를 예측 (t+1의 인과성 학습)
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 128),
            nn.BatchNorm1d(128),
            nn.Tanh(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(128, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 512, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 1)
        )

        self.concat7_shape = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            
        )
        
        self.concat6_shape = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            
        )
        
        self.concat5_shape = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            
        )
        
        self.concat4_shape = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            
        )
        
        self.concat3_shape = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            
        )
        
        self.concat2_shape = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            
        )
        
        self.concat1_shape = nn.Sequential(
            nn.Conv1d(in_channels = 1024, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            
        )
        
        
    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x1 = self.conv1(x)
        #print(x1.shape)
        x = F.pad(x1, pad = (4, 0, 0, 0))
        x2 = self.conv2(x)
        #print(x2.shape)
        x = F.pad(x2, pad = (8, 0, 0, 0))
        x3 = self.conv3(x)
        #print(x3.shape)
        x = F.pad(x3, pad = (16, 0, 0, 0))
        x4 = self.conv4(x)
        #print(x4.shape)
        x = F.pad(x4, pad = (32, 0, 0, 0))
        x5 = self.conv5(x)
        #print(x5.shape)
        x = F.pad(x5, pad = (64, 0, 0, 0))
        x6 = self.conv6(x)
        #print(x6.shape)
        x = F.pad(x6, pad = (128, 0, 0, 0))
        x7 = self.conv7(x)
        #print(x7.shape)
        encode = self.encoder_fc(x7.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        x = torch.cat([x7,x],1) # 4,16,7501
        x = self.concat7_shape(x) 
        x = F.pad(x, pad = (130, 0, 0, 0)) 
        x = self.t_conv1(x) # 4,8,7501
        #print(x.shape,'new_cat7')
        
        
        x = torch.cat([x6,x],1)
        x = self.concat6_shape(x)
        x = F.pad(x, pad = (66, 0, 0, 0))
        x = self.t_conv2(x)
        
        
        x = torch.cat([x5,x],1)
        x = self.concat5_shape(x)
        x = F.pad(x, pad = (34, 0, 0, 0))
        x = self.t_conv3(x)
        
        
        
        x = torch.cat([x4,x],1)
        x = self.concat4_shape(x)
        x = F.pad(x, pad = (18, 0, 0, 0))
        x = self.t_conv4(x)
        
        
        x = torch.cat([x3,x],1)
        x = self.concat3_shape(x)
        x = F.pad(x, pad = (10, 0, 0, 0))
        x = self.t_conv5(x)
        
        
        
        x = torch.cat([x2,x],1)
        x = self.concat2_shape(x)
        x = F.pad(x, pad = (6, 0, 0, 0))
        x = self.t_conv6(x)
        
        
        
        x = torch.cat([x1,x],1)
        x = self.concat1_shape(x)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv7(x)
    
        
        #print(x.shape)
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x
'''

"\n# skip connection\n# encoder <- 과거의 데이터를 바탕으로 미래를 예측 (t-1의 인과성 학습)\n# decoder <- 미래의 데이터를 바탕으로 과거를 예측 (t+1의 인과성 학습)\nclass TimeAutoEncoder(nn.Module):\n    def __init__(self):\n        super(TimeAutoEncoder, self).__init__()\n        self.conv1 = nn.Sequential(\n            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),\n            nn.BatchNorm1d(512),\n            nn.ReLU(),\n        )\n\n        self.conv2 = nn.Sequential(\n            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),\n            nn.BatchNorm1d(256),\n            nn.ReLU(),\n        )\n\n        self.conv3 = nn.Sequential(\n            nn.Conv1d(in_channels = 256, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 4),\n            nn.BatchNorm1d(128),\n            nn.ReLU(),\n        )\n\n        self.conv4 = nn.Sequential(\n            nn.Conv1d(in_channels = 128, out_channels 

In [9]:
model = TimeAutoEncoder().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.MSELoss()

In [11]:
#!pip install torchmetrics
#from torchmetrics import MeanAbsolutePercentageError

#model = TimeAutoEncoder().to(DEVICE)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
#criterion = MeanAbsolutePercentageError()


In [10]:
gc.collect()

26

In [11]:
import time

model_dir = '/home/work/Tcae_apply/model_dir3/'
min_loss = 987654321

epochs = 150
batch_size = 64

train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)

#tloss_list=[]
#vloss_list=[]

for epoch in tqdm(range(1, epochs + 1)):
    start = time.time()
    
    
    train_loss = train(model = model, train_loader = train_batch_li) 
    val_loss = val(model = model, train_loader = val_batch_li) 
    end = time.time()
    
    
    #tloss_list.append(train_loss)
    #vloss_list.append(val_loss)
    
    print(f'EPOCH:{epoch}, Train Loss:{train_loss}, Val Loss:{val_loss}, 학습 시간: {end - start}')
    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model.state_dict(), model_dir + f'TimeAutoEncoder_skipconnection_val.pt')
        print('모델 저장')
        
    

  0%|          | 0/150 [00:00<?, ?it/s]

EPOCH:1, Train Loss:147529.8782110699, Val Loss:15281.2935546875, 학습 시간: 81.71810507774353
모델 저장
EPOCH:2, Train Loss:10662.40383342161, Val Loss:5051.080322265625, 학습 시간: 32.199466705322266
모델 저장
EPOCH:3, Train Loss:3710.6201958090573, Val Loss:2342.2825317382812, 학습 시간: 26.571794033050537
모델 저장
EPOCH:4, Train Loss:2031.1439271054026, Val Loss:1485.2213256835937, 학습 시간: 26.208123445510864
모델 저장
EPOCH:5, Train Loss:1347.0279706534693, Val Loss:1096.1991455078125, 학습 시간: 26.76035237312317
모델 저장
EPOCH:6, Train Loss:1031.0193729723915, Val Loss:989.2414001464844, 학습 시간: 26.018693447113037
모델 저장
EPOCH:7, Train Loss:944.8359095686573, Val Loss:743.947802734375, 학습 시간: 26.34234070777893
모델 저장
EPOCH:8, Train Loss:694.6019990565413, Val Loss:609.6730773925781, 학습 시간: 26.398714065551758
모델 저장
EPOCH:9, Train Loss:595.6984164997683, Val Loss:529.0238616943359, 학습 시간: 25.81749153137207
모델 저장
EPOCH:10, Train Loss:663.3840906175517, Val Loss:702.9898498535156, 학습 시간: 26.375518321990967
EPOCH:11, Trai

EPOCH:86, Train Loss:63.71654853174242, Val Loss:73.27498092651368, 학습 시간: 25.833107233047485
EPOCH:87, Train Loss:58.52097540386652, Val Loss:37.254844665527344, 학습 시간: 25.87941551208496
EPOCH:88, Train Loss:34.00888164972855, Val Loss:34.54324836730957, 학습 시간: 25.918862104415894
EPOCH:89, Train Loss:43.75223127462096, Val Loss:48.19899291992188, 학습 시간: 25.84086513519287
EPOCH:90, Train Loss:39.536511308055815, Val Loss:41.66391315460205, 학습 시간: 25.83341073989868
EPOCH:91, Train Loss:39.7438505463681, Val Loss:42.51291027069092, 학습 시간: 25.825258016586304
EPOCH:92, Train Loss:32.36233151969263, Val Loss:30.449234580993654, 학습 시간: 25.816336393356323
EPOCH:93, Train Loss:25.936863769919185, Val Loss:23.198488235473633, 학습 시간: 25.79295778274536
모델 저장
EPOCH:94, Train Loss:33.453416113126075, Val Loss:21.28130226135254, 학습 시간: 25.73160433769226
모델 저장
EPOCH:95, Train Loss:25.29815187292584, Val Loss:22.12858352661133, 학습 시간: 25.72589111328125
EPOCH:96, Train Loss:21.77077626373808, Val Loss:

In [ ]:
#model = TimeAutoEncoder().to(DEVICE)
#model.load_state_dict(torch.load(model_dir + f'TimeAutoEncoder_val.pt', map_location = DEVICE))

In [ ]:
#mel_embeding_li = get_mel_embeding(model = model, train_loader = test_batch_li)
#mel_embeding = np.concatenate(mel_embeding_li, axis = 0)

In [ ]:
#np.save(batch_data_dir + 'mel_embeding_val.npy', mel_embeding)